In [1]:
import string
import random
source = string.ascii_lowercase + string.ascii_uppercase
teststring = 'abccdeefg'
teststring = ''.join(random.choices(source,k=100000000))
b_teststring = teststring.encode()

In [2]:
def count_doubles_once(val):  # from article
    total = 0
    chars = iter(val)
    c1 = next(chars)
    for c2 in chars:
        if c1 == c2:
            total += 1
        c1 = c2
    return total

# uint64_t count_byte_doubles(char * str) {
#   uint64_t count = 0;
#   while (str[0] && str[1]) {
#     if (str[0] == str[1]) count++;
#     str++;
#   }
#   return count;
# }

In [3]:
%load_ext cython

In [4]:
%%cython
#cython: language_level=3, boundscheck=False, wraparound=False, cdivision=True
cimport cython

cpdef int c_count(char * str):
    cdef int i
    cdef int count = 0
    for i in range(len(str)-1):
        if str[i] == str[i+1]:
            count += 1
    return count

cpdef int c_count2(char * str):
    cdef int count = 0, i = 1
    cdef int c0 = str[0], c = str[1]
    while c:
        if c0 == c:
            count += 1
        i += 1
        c0 = c
        c = str[i]        
    return count

In [5]:
%%cython
#cython: language_level=3, boundscheck=False, wraparound=False, cdivision=True
from cython.parallel import parallel, prange

cpdef int c_count3(char * str):
    cdef int count = 0, i = 1
    cdef int size = len(str)-1
    for i in prange(size, num_threads=4, nogil=True):
        if str[i] == str[i+1]:
            count += 1
    return count

In [7]:
import time

val = 0
t0 = time.time()
val = count_doubles_once(teststring)
print(time.time() - t0, val)

t0 = time.time()
val = c_count(b_teststring)
print(time.time() - t0, val)

t0 = time.time()
val = c_count2(b_teststring)
print(time.time() - t0, val)

t0 = time.time()
val = c_count3(b_teststring)
print(time.time() - t0, val)

print(3.5265986919403076 / 0.09255337715148926, "for range(len)")
print(3.5265986919403076 / 0.08615541458129883, "while")
print(3.5265986919403076 / 0.021694183349609375, "parallel, prange(len)")

3.4526546001434326 1924295
0.09293532371520996 1924295
0.08688235282897949 1924295
0.0208432674407959 1924295
38.103403684211884 for range(len)
40.93298963366375 while
162.55964260594337 parallel, prange(len)
